In [69]:
# this is grouped activity watch data
aw_data = pd.read_csv("data/aw_data_grouped.csv")

# filter for neurotech events
import re

def is_neuro(text):
    if re.search('(neuro)|(eeg)|(brain)',text) != None:
        return 1
    return 0

aw_data["is_neurotech"] = aw_data["event_classification"].apply(lambda x: is_neuro(x))

# set duration second to zero if it is not neuro
aw_data["duration_seconds"] = aw_data[['is_neurotech','duration_seconds']].apply(lambda x: x['duration_seconds'] if x['is_neurotech'] == 1 else 0, axis=1)

# get data summarized by day
aw_data_neuro_grouped = aw_data.groupby(["summary_day"], as_index=False)
aw_data_neuro_grouped = aw_data_neuro_grouped.agg({
                            'is_neurotech': 'sum',
                            'duration_seconds': 'sum'
                        })
aw_data_neuro_grouped.rename(columns = {'is_neurotech':'activity_frequency', 'duration_seconds':'activity_duration'}, inplace = True)
aw_data_neuro_grouped

,summary_day,activity_frequency,activity_duration
0,2021-02-27,4,60.327
1,2021-02-28,3,2775.748
2,2021-03-01,4,3952.435
3,2021-03-02,2,392.237
4,2021-03-03,1,15.462
5,2021-03-04,0,0.000
6,2021-03-05,0,0.000
7,2021-03-06,2,771.974
8,2021-03-07,3,541.685
9,2021-03-08,2,59.005


In [78]:
oura_data = pd.read_csv("data/oura_cleaned.csv")

print(oura_data.columns)

Index(['Unnamed: 0', 'average_met', 'cal_active', 'cal_total_kcal',
       'class_5min', 'daily_movement', 'day_end', 'day_start', 'high_minutes',
       'inactive_minutes', 'inactivity_alerts', 'low_minutes',
       'medium_minutes', 'met_1min', 'met_min_high', 'met_min_inactive',
       'met_min_low', 'met_min_medium', 'non_wear_minutes', 'rest_minutes',
       'rest_mode_state_act_1', 'score_act_1', 'score_meet_daily_targets',
       'score_move_every_hour', 'score_recovery_time', 'score_stay_active',
       'score_training_frequency', 'score_training_volume', 'steps',
       'summary_date_act', 'target_calories', 'target_km', 'target_miles',
       'timezone_act', 'to_target_km', 'to_target_miles', 'total_act_1',
       'bedtime_end_act_1', 'bedtime_start_act_1', 'breath_average_act_1',
       'duration_act_1', 'period_id_act_1', 'summary_date_res', 'timezone_res',
       'hr_average_act_1', 'hr_lowest_act_1', 'rmssd_act_1', 'period_id_res_1',
       'rest_mode_state_res_1', 'score

In [81]:
# create combined dataset for correlation
# data column here is - summary_day
aw_for_correlation = aw_data_neuro_grouped.iloc[:-1]
aw_for_correlation.set_index('summary_day')

# date column here is - summary_date_act (could be different we have like 4 candidates in this dataset)
oura_data_for_correlation = oura_data.iloc[38:, ]
oura_data_for_correlation.rename(columns = {'summary_date_act':'summary_day'}, inplace = True)
oura_data_for_correlation.set_index("summary_day")


combined_dataset = aw_for_correlation.merge(oura_data_for_correlation, on="summary_day")
combined_dataset

,summary_day,activity_frequency,activity_duration,Unnamed: 0,average_met,cal_active,cal_total_kcal,class_5min,daily_movement,day_end,...,score_efficiency,score_latency,score_rem,score_total,summary_date_res_1,temperature_delta_celsius,temperature_deviation,timezone,total_res_1,temperature_trend_deviation
0,2021-02-27,4,60.327,38,1.31250,190,2267,222232321122232223322223333222222222222223333222222222222222223232322322232222222232222221011111121122111111111112222223322332333333222322323333333332222222222222222222222222221100003222232222222222222223221121222211121112222211222222222222222222122221122111112222222222111111111111111111,3288,2021-02-28T03:59:59-08:00,...,74.0,75.0,9.0,43.0,2021-03-05,-0.03,-0.03,-480.0,18090.0,0.15
1,2021-02-28,3,2775.748,39,1.28125,230,2264,111111111111111112220000000000000000034322322232233233333222222223332232333222322222221123222212223323344322111111212322223332322222222112222222233332222222222222222223232232222111122220000000000000000000000000002211111111111111111111111111111111111111111111111111111111111111111111111111,4015,2021-03-01T03:59:59-08:00,...,88.0,91.0,8.0,27.0,2021-03-06,-0.01,-0.01,-480.0,14040.0,0.07
2,2021-03-01,4,3952.435,40,1.43750,422,2524,111111111111212222224353342343333222223333233322322222322232222222222222222222222222222333343222222222222222222222222222222221332232222222222222123335433443333222222122222332334332332222222222222222222332222221123111111111111111111111111111111111111111111111111111111111111111112211111111,7605,2021-03-02T03:59:59-08:00,...,93.0,67.0,41.0,41.0,2021-03-08,-0.16,-0.16,-480.0,17640.0,0.25
3,2021-03-02,2,392.237,41,1.31250,182,2251,111111112111111112211122222222122111222333323223343331003334222222233322222222222222222222222222333333322222222222222222222222222222222222222222223232222222222222100000000000000112122222322100000000000222222223222222222222221212222222111111122212323222200000000011111111111111111111111111,3317,2021-03-03T03:59:59-08:00,...,98.0,67.0,25.0,51.0,2021-03-09,-0.02,-0.02,-480.0,20340.0,0.13
4,2021-03-03,1,15.462,42,1.50000,444,2574,111111121111122212234333104444433322222222222222222222323322222222222232332232322233322222223222212222322222222223443222233442222222222222332233332323223332222222222222232222222222222223333222222222222222234443343222222222333432222211111111111111111111111111111111111111111111111111111111,8062,2021-03-04T03:59:59-08:00,...,96.0,89.0,12.0,28.0,2021-03-10,-0.32,-0.32,-480.0,14130.0,-0.08
5,2021-03-04,0,0.000,43,1.46875,394,2535,111111111111111122222222332123323333233333333222222222233232222222335432222222222222223222332322222222222222223332432323222222222222222222322332223332222233332232222222333322232222334333322222222222222222322222222222222322223211111111111111111111111111111111111111111111111111111111111111,7165,2021-03-05T03:59:59-08:00,...,81.0,94.0,96.0,98.0,2021-03-11,0.13,0.13,-480.0,32730.0,0.00
6,2021-03-05,0,0.000,44,1.59375,572,2725,121111112111122333542233343333322222222222222234444434443233222222222222222222322222222222232332222233222222342333322223222222233443222222332223222222221233322322222233222222222323222223233322223332233222211211111122111111111111111111222222222222222222223322222222211111211222111111111111,10355,2021-03-06T03:59:59-08:00,...,74.0,91.0,33.0,44.0,2021-03-12,0.16,0.16,-480.0,18480.0,0.13
7,2021-03-06,2,771.974,45,1.15625,68,2021,111111111111111112311111111111111111111122222333223322222222222222333322222222222221111111111111111111111111111112222211121222221111111112211122222233000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,1200,2021-03-07T03:59:59-08:00,...,88.0,75.0,42.0,71.0,2021-03-13,0.31,0.31,-480.0,25500.0,0.30
8,2021-03-07,3,541.685,46,1.34375,307,2319,000000000000000000000000000000000000000000000000000000000000000000000003322222222222211111111111111111111111111112222112222242223234333344444333222222222222332222223223343222221111111111111121222112223343333322222222222242